# Compiler Equational



# Imp
https://softwarefoundations.cis.upenn.edu/lf-current/Imp.html


In [1]:
from kdrag.all import *
Aexp = kd.Inductive("Aexp", strict=False)
Aexp.declare("num", ("n", smt.IntSort()))
Aexp.declare("add", ("add1", Aexp), ("add2", Aexp))
Aexp = Aexp.create()

a,b = smt.Consts("a b", Aexp)
aeval = smt.Function("aeval", Aexp, smt.IntSort())
aeval = kd.define("aeval", [a],
                  kd.cond((a.is_num, a.n),
                          (a.is_add, aeval(a.add1) + aeval(a.add2))))
n = smt.Const("n", smt.IntSort())

#aeval_num = kd.lemma(smt.ForAll([n], aeval(Aexp.num(n)) == n))
#kd.kernel.defns[aeval]
aeval_num = kd.lemma(smt.ForAll([n], aeval(Aexp.num(n)) == n), by=aeval.defn)
aeval_add = kd.lemma(smt.ForAll([a], aeval(Aexp.add(a,b)) == aeval(a) + aeval(b)), by=aeval.defn)

optimize = smt.Function("optimize", Aexp, Aexp)
optimize = kd.define("optimize", [a], kd.cond((a.is_add, 
                                                smt.If(smt.And(a.add1.is_num, a.add1.n == 0), 
                                                       a.add2,
                                                       Aexp.add(optimize(a.add1), optimize(a.add2)))),
                                              (a.is_num, a)))




Admitting lemma ForAll([x, y, z], mul(x, mul(y, z)) == mul(mul(x, y), z))


Admitting lemma ForAll(x, Implies(x >= 0, sqrt(x) >= 0))
Admitting lemma ForAll(x, Implies(x >= 0, sqrt(x)**2 == x))
Admitting lemma ForAll(x, Implies(x >= 0, sqrt(sqr(x)) == x))


In [17]:
l = kd.Lemma(smt.ForAll([a], aeval(optimize(a)) == aeval(a)))
_a = l.fixes()
l.induct(_a)
"""
try:
    while True:
        l.auto(by=[aeval.defn, optimize.defn])
except _ as e:
    pass
l
"""
_n = l.fixes()
l.unfold(aeval, optimize)
l.simp()
l.unfold(aeval)
l.simp()
l.auto()

_a1 = l.fixes()
l.intros()
_a2 = l.fixes()
l.intros()
l.unfold(aeval)
l.simp()
l.unfold(optimize)
l.simp()
l.cases(smt.And(_a2.is_num, _a2.n == 0))

l.rw(2)
l.simp()
l.auto(by=[aeval.defn])

l.rw(2)
l.simp()
l.auto(by=[aeval.defn])

l.qed()


|- ForAll(a, aeval(optimize(a)) == aeval(a))

Ok. Having repeat wopuld be nice.
Again, induction not mangling would be nice.
Choosing which thing to unfold was tough.
These are unreadable.
I don't really know how to do forward proofs.
Not being able to pick the names is tough.



In [22]:
l = kd.Lemma(smt.ForAll([a], aeval(optimize(a)) == aeval(a)))
_a = l.fixes()
l.induct(_a)
_a = l.fixes()
l.unfold(aeval, optimize)
l.auto(by=[aeval.defn, optimize.defn])

l.unfold(aeval,optimize)
l.cases(smt.And(_a.is_add, _a.add1.n == 0))


[] ?|- ForAll(a!689,
       Or(Not(aeval(optimize(a!689)) == aeval(a!689)),
          ForAll(a!688,
                 Or(Not(aeval(optimize(a!688)) ==
                        aeval(a!688)),
                    aeval(optimize(add(a!688, a!689))) ==
                    aeval(add(a!688, a!689))))))

# Hoare Logic
Separation logic. Not going to play that nice with z3.

Hmm. The raw cvc5 interface isn't that pleasant.

Winskel
Software Foundations
Dafny Book
Frama-C

Use Weepy?



    



```python
from z3 import *

# Yeah. Is there a point to this? Should I just use python datatypes?

Imp = Datatype("Imp")
Imp.declare("skip")
Imp.declare("seq", ("left", Imp), ("right", Imp))
Imp.declare("assign", ("var", StringSort()), ("expr", IntSort())) # it is eaisest if expr is a z3 expr.
Imp = Imp.create()
Imp.assign(StringVal("x"), 1)# IntVal(1))
x = StringVal("x")
prog = Imp.seq(Imp.assign(x, 1), 
               Imp.assign(x, 2))

# This doesn't work (?) because I need to also embed the logic in itself. Meh.
# or   ArraySort(Rho,BoolSort()) # i did this in a pure smtlib example once.
# Oh actually this does play in with quote.


#Triple = Datatype("Triple")
#Triple.declare("triple", ("pre", IntSort()), ("cmd", Imp), ("post", IntSort()))
from collections import namedtuple
Triple = namedtuple("Triple", ["pre", "cmd", "post", "proof"])
# or two datatypes Triple and Proof(triple,pf)


# wp as a hoare axiom schema.
def wp(cmd, post) -> Triple:
    def wp(cmd, post):
        if cmd.sort().name() == "assign":
            return post.substitute(cmd.var, cmd.expr)
        elif cmd.sort().name() == "seq":
            return wp(cmd.left, wp(cmd.right, post))
        elif cmd.sort().name() == "skip":
            return post
        else:
            raise Exception("Unknown command: %s" % cmd)
    return Triple(wp(cmd,post), cmd, post, "wp_ax")

def seq(trip1 : Triple, trip2: Triple, by=[]) -> Triple:
    pre1, cmd1, post1 = trip1
    pre2, cmd2, post2 = trip2.cmd
    prove(Imp(And(by, post1), pre2))
    return Triple(pre1, Imp.seq(cmd1,cmd2), post2)

def while_(inv, body:Triple, by=[]):
    pass



# hmm. Maybe an tree deconstruction of different kinds might lead to different proof systems of cfg systems.

```


```python
import cvc5
solver = cvc5.Solver()
solver.setLogic("ALL")
realSort = solver.getRealSort()
constraint1 = solver.mkTerm(Kind.LT, zero, x)
solver.assertFormula(constraint4)
solver.resetAssertions()
r2 = solver.checkSat()
unsatCore = solver.getUnsatCore();
# https://greentreesnakes.readthedocs.io/en/latest/nodes.html 
import ast
tree = ast.parse("x = x + 3", mode="single")
match tree.body[0]:
    case ast.Assign([ast.Name(name, ast.Store())], expr):
        print(name, expr)
    case x:
        print(x)


def wp(stmt,post):
    match stmt:
        case ast.Assign([ast.Name(name, ast.Store())], expr):
            return z3.substitute(z3.Int("name"), expr)
        case ast.Pass():
            return post
        case _:
            assert False

def triple(pre,stmt,post):
    mywp = wp(stmt,post)

def assign_triple(pre : Form, name, expr, post : Form, by=[]):
    wp = z3.substitute(post, (x, expr))
    check(infer(by, pre > wp))
    return trust( (pre, ast.Assign([name], expr), post))

def seq_triple(s1 : Triple, s2: Triple):
```


#  Nand2Knuck

Mike Gordon https://arxiv.org/abs/1806.04002 obit
use shallow HOL for circuits
shallow hol for hoarse logic? State -> State -> Bool


https://www.nand2tetris.org/course

https://www.philipzucker.com/nand2tetris-chc/
https://www.philipzucker.com/nand2tetris-cpu/
https://www.philipzucker.com/nand2tetris-in-verilog-and-fpga-and-coq/
https://github.com/philzook58/nand2coq

In [6]:
Nandtable = [
    [1, 1, 0],
    [1, 0, 1],
    [0, 1, 1],
    [0, 0, 1]
]

def test_nand():
    for x,y,z in Nandtable:
        assert not (x and y) == z

test_nand()

def fuzz_nand():
    for testnum in range(1000):
        x = random.randint()
        y = random.randint()
        z = nand(x, y)



SyntaxError: invalid syntax (3501921742.py, line 12)

In [1]:
import knuckledragger as kd
from knuckledragger import lemma, axiom, define

from z3 import *

B = BoolSort()
BV1 = BitVecSort(1)
x,y,z,w,u,v = Consts("x y z w u v", BV1)
"""
CNand = Function("Nand", B,B,B,B)
nand_def = axiom(ForAll([x,y,z], CNand(x,y,z) == 
     Or(And(x == True,  y == True,  z == False),
        And(x == True,  y == False, z == True),
        And(x == False, y == True,  z == True),
        And(x == False, y == False, z == True))))
"""

CNand = define("CNand", [x,y,z], Or(
    And(x == 1, y == 1, z == 0),
    And(x == 1, y == 0, z == 1),
    And(x == 0, y == 1, z == 1),
    And(x == 0, y == 0, z == 1)))


test1 = lemma(CNand(True,True,False), by=[CNand.defn])

nand_fun = lemma(ForAll([x,y,z], CNand(x,y,z) == (~(x & y) == z)), by=[CNand.defn])
nand_fun

|- ForAll([x, y, z], CNand(x, y, z) == (~(x & y) == z))

In [2]:
test2 = lemma(CNand(True,True,True), by=[CNand.defn])

LemmaError: (CNand(1, 1, 1), 'Countermodel', [knuckledragger_goal = True,
 by_0 = True,
 CNand = [(1, 1, 1) -> False,
          else ->
          Or(Not(Or(Not(Var(1) == 1),
                    Not(Var(2) == 0),
                    Not(Var(0) == 1))),
             Not(Or(Not(Var(1) == 0),
                    Not(Var(0) == 1),
                    Not(Var(2) == 1))),
             Not(Or(Not(Var(1) == 1),
                    Not(Var(2) == 1),
                    Not(Var(0) == 0))),
             Not(Or(Not(Var(1) == 0),
                    Not(Var(2) == 1),
                    Not(Var(0) == 0))))]])

In [3]:
#CNot = Function("CNot", B, B, B)
#not_def = ForAll([x,y], CNot(x,y) == CNand(x,x,y))

CNot = define("CNot", [x,y], CNand(x,x,y))
lemma(ForAll([x,y], CNot(x,y) == (~x == y)), by=[CNand.defn, CNot.defn])

cnot_fun = lemma(ForAll([x,y], CNot(x,y) == (~x == y)), by=[nand_fun, CNot.defn])


CAnd = define("CAnd", [x,y,z], Exists([w], And(CNand(x,y,w), CNot(w,z))))

cand_fun = lemma(ForAll([x,y,z], CAnd(x,y,z) == (x & y == z)), by=[cnot_fun, nand_fun, CAnd.defn])


COr = define("COr", [x,y,z], Exists([w,u], And(CNot(x,w), CNot(y,u), 
                                                   CNand(w,u,z))))

cor_fun = lemma(ForAll([x,y,z], COr(x,y,z) == ((x | y) == z)), by=[cnot_fun, nand_fun, COr.defn])

a,b,out,sel,sela,selb,notsel = Consts("a b out sel sela selb notsel", BV1)
Mux = define("Mux", [a,b,sel,out], Exists([sela,selb,notsel], And(
    CNot(sel, notsel), 
    CAnd(notsel, a, sela),
    CAnd(sel, b, selb),
    COr(sela, selb, out))))

mux_fun = lemma(ForAll([a,b,sel,out], Mux(a,b,sel,out) == (out == If(sel == 0, a, b))), by=[Mux.defn, cnot_fun, cand_fun, cor_fun])



In [18]:
BV1 = BitVecSort(1)
BV2 = BitVecSort(2)
BV4 = BitVecSort(4)
BV8 = BitVecSort(8)
BV16 = BitVecSort(16)

#bool2bv = define("bool2bv", [x], If(x, BitVecVal(1,1), BitVecVal(0,1)))
#bv2bool = define("bv2bool", [x], x != BitVecVal(0,1))

# but maybe we hsould have never done Bool in the first place.
x2,y2,z2 = Consts("x2 y2 z2", BV2)
CNand2 = define("CNand2", [x2,y2,z2], CNand(Extract(0,0,x2), Extract(0,0,y2), Extract(0,0,z2)) &
                                    CNand(Extract(1,1,x2), Extract(1,1,y2), Extract(1,1,z2)))

nand2_fun = lemma(ForAll([x2,y2,z2], CNand2(x2,y2,z2) == (~(x2 & y2) == z2)), by=[nand_fun, CNand2.defn])







## Chapter 2


In [ ]:
half_adder = define("HalfAdder", [a,b,c,s], 
    Or(
        And(a == False, b == False, c == False, s == False),
        And(a == False, b == True,  c == False, s == True),
        And(a == True,  b == False, c == False, s == True),
        And(a == True,  b == True,  c == True,  s == False)
    ))


# FullAdder


# RippleCarryAdder


# ALU


https://lawrencecpaulson.github.io/2023/01/04/Hardware_Verification.html
https://www.cl.cam.ac.uk/techreports/UCAM-CL-TR-77.html Why higher-order logic is a good formalisation for specifying and verifying hardware

Use signals. Implicit clock.

relational frp

In [ ]:
def Signal(T): # a sequence
     return ArraySort(IntSort(), T)
# tla plus

flipflip = kd.define("flipflip", [in1, reset, out1], 
                     ForAll([t], If(reset[t], out1[t+1] == 0, out1[t+1] == in1[t])))


def lift(f):
    """ lift combinatorial relation to a signal relation"""
    args = FreshConst(Signal(f.domain(i))) for i in f.arity()
    return Lambda(args, ForAll([t], f(*[arg[t] for arg in args])))

And16 = kd.define("And16", lift(And16))

In [ ]:

#yada yada


flipflip == Function("D",B,B,B,B)
flip_flip_def = ForAll([x,y])


## CPU Inns
use simp as slow evaluator.



In [1]:
from z3 import *
import knuckledragger as kd
Comp, comp = EnumSort("Comp",  ["ZERO", "ONE", "NEGONE", "D", "A", "M", "NOTD", "NOTA", "NOTM", "NEGD", "NEGA", "NEGM", "INCD", "INCA", "INCM", "DECD", "DECA", "DECM", "ADD_DA", "ADD_DM", "SUB_DA", "SUB_DM", "SUB_AD", "SUB_MD", "AND_DA", "AND_DM", "OR_DA", "OR_DM"])
Dest, dest = EnumSort("Dest", ["NULL", "M", "D", "MD", "A", "AM", "AD", "AMD"])
Jump, jump = EnumSort("Jump", ["NULL", "JGT", "JEQ", "JGE", "JLT", "JNE", "JLE", "JMP"])

In [11]:
Insn = Datatype("Insn")
Insn.declare("A", ("lit", BitVecSort(15)))
Insn.declare("C", ("comp", Comp), ("dest", Dest), ("jump", Jump))
Insn = Insn.create()

BV16 = BitVecSort(16)
CPUState = kd.notation.Record("CPUState", ("mem", ArraySort(BV16, BV16)), ("pc", BV16), ("a", BV16), ("d", BV16))




In [17]:
i = Const("i", Insn)
st = Const("st", CPUState)
c = kd.notation.Cond()
#c.when(i.is_C).then(Cond().when(i.comp.is_ZERO).then())
c.when(i.is_A).then(CPUState.mk(st.mem, st.pc, Concat(BitVecVal(0, 1), i.lit), st.d))

c.when(i.is_C).then(kd.Cond()
                    cond(i.comp.is_ZERO, CPUState.mk(st.mem, st.pc, st.a, st.d))
                    .when().then().CPUState.mk(st.mem, st.pc, st.a, st.d))
e = c.otherwise(st)
kd.define("exec_insn", [st, i], e)

       exec_insn(st, i) ==
       If(is(A, i),
          mk(mem(st), pc(st), Concat(0, lit(i)), d(st)),
          st)) to ForAll([st, i],
       exec_insn(st, i) ==
       If(is(A, i),
          mk(mem(st), pc(st), Concat(0, lit(i)), d(st)),
          If(is(C, i),
             mk(mem(st), pc(st), a(st), d(st)),
             st)))


exec_insn

In [9]:
z3.Z3_DEBUG = True

In [10]:
MyType = Datatype('MyType')
MyType.declare('mk_mytype', ('children', ArraySort(IntSort(), MyType)))
MyType = MyType.create()

Z3Exception: Z3 sort expected

In [7]:
Vec3 = z3.Datatype("Vec3")
Vec3.declare("Vec3__init",
  ("x", z3.IntSort()),
  ("y", z3.IntSort()),
  ("z", z3.IntSort())
)
Vec3 = Vec3.create()

PointerVec3 = z3.Datatype("Pointer__Vec3")
PointerVec3.declare("Pointer__Vec3__init",
  ("data", z3.ArraySort(z3.BitVecSort(32), Vec3)),
  ("nindices", z3.IntSort()),
  ("indices", z3.ArraySort(z3.IntSort(), z3.BitVecSort(32)))
)
PointerVec3 = PointerVec3.create()

In [ ]:
from dataclasses import dataclass
from enum import Enum       
import typing
from typing import List, Union
 
Comp = Enum('Comp', '''ZERO ONE NEGONE D A M NOTD NOTA NOTM NEGD NEGA NEGM INCD INCA INCM DECD DECA DECM 
                         ADD_DA ADD_DM SUB_DA SUB_DM SUB_AD SUB_MD AND_DA AND_DM OR_DA OR_DM''') 
Dest = Enum('Dest', 'NULL M D MD A AM AD AMD') 
Jump = Enum('Jump', 'NULL JGT JEQ JGE JLT JNE JLE JMP') 
 
@dataclass
class Ainstr:
    lit: int
 
@dataclass
class Cinstr:
    dest: Dest
    comp: Comp
    jump: Jump
 
@dataclass
class HackState:
    rA: int
    rD: int
    pc: int
    mem: List[int]
 
def execute(instr : Cinstr, s : HackState):
    A = s.rA
    D = s.rD
    pc = s.pc
    M = s.mem[A]
    
    if instr.comp == Comp.ZERO:
        comp = 0
    elif instr.comp == Comp.ONE:
        comp = 1
    elif instr.comp == Comp.NEGONE:
        comp = -1
    elif instr.comp == Comp.D:
        comp = D
    elif instr.comp == Comp.A:
        comp = A
    elif instr.comp == Comp.M:
        comp = M
    elif instr.comp == Comp.NOTD:
        comp = ~D
    elif instr.comp == Comp.NOTA:
        comp = ~A
    elif instr.comp == Comp.NOTM:
        comp = ~M
    elif instr.comp == Comp.NEGD:
        comp = -D
    elif instr.comp == Comp.NEGA:
        comp = -A
    elif instr.comp == Comp.NEGM:
        comp = -M
    elif instr.comp == Comp.INCD:
        comp = D + 1
    elif instr.comp == Comp.INCA:
        comp = A + 1
    elif instr.comp == Comp.INCM:
        comp = M + 1
    elif instr.comp == Comp.DECD:
        comp = D - 1
    elif instr.comp == Comp.DECA:
        comp = A - 1
    elif instr.comp == Comp.DECM:
        comp = M - 1
    elif instr.comp == Comp.ADD_DA:
        comp = D + A
    elif instr.comp == Comp.ADD_DM:
        comp = D + M
    elif instr.comp == Comp.SUB_DA:
        comp = D - A
    elif instr.comp == Comp.SUB_DM:
        comp = D - M
    elif instr.comp == Comp.SUB_AD:
        comp = A - D
    elif instr.comp == Comp.SUB_MD:
        comp = M - D
    elif instr.comp == Comp.AND_DA:
        comp = D & A
    elif instr.comp == Comp.AND_DM:
        comp = D & M
    elif instr.comp == Comp.OR_DA:
        comp = D | S
    elif instr.comp == Comp.OR_DM:
        comp = D | M
    else:
        assert False
    
    if instr.dest == Dest.NULL:
        pass
    elif instr.dest == Dest.M:
        M = comp
    elif instr.dest == Dest.D:
        D = comp
    elif instr.dest == Dest.MD:
        M = comp
        D = comp
    elif instr.dest == Dest.A:
        A = comp
    elif instr.dest == Dest.AM:
        A = comp
        M = comp
    elif instr.dest == Dest.AD:
        A = comp
        D = comp
    elif instr.dest == Dest.AMD:
        A = comp
        M = comp
        D = comp
    else:
        assert False
        
    if instr.jump == Jump.NULL:
        jump = False
    elif instr.jump == Jump.JGT:
        jump = comp > 0
    elif instr.jump == Jump.JEQ:
        jump = comp == 0
    elif instr.jump == Jump.JGE:
        jump = comp >= 0
    elif instr.jump == Jump.JLT:
        jump = comp < 0
    elif instr.jump == Jump.JNE:
        jump = comp != 0
    elif instr.jump == Jump.JLE:
        jump = comp <= 0
    elif instr.jump == Jump.JMP:
        jump = True
    else:
        assert False
    
    # we may not want to mutate here.
    # return comp, jump, dest instead?
    if jump:
        s.pc = s.rA
    else:
        s.pc = pc + 1
    
    s.mem[s.rA] = M #??? What is the order here?
    s.rA = A
    s.rD = D
 
    # return A, D, M, jump ?
    return s
 
Instr = Union[Ainstr,Cinstr]
 
def fetch(rom : List[Instr], s : HackState) -> Instr:
    return rom[s.pc]
 
def run(rom):
    # hmm. We should make 16 bit vectors.
    state = HackState(0,0,0,[0]*2**13)
    while state.pc != -1:
        instr = fetch(rom, state)
        state = execute(instr, state)
    return state

# Extract
In principle, extraction to verilog isn't that hard?
We need to identify input and output.
clk is implicit threaded everywhere
flipflip is primitive




# TD4 
verilog
vhdl
knuckle



# Other
Calculating compilers
https://xavierleroy.org/courses/EUTypes-2019/


In [2]:
def Switch(*cases):
    if len(cases) == 1:
        return cases[0]
    assert len(cases) % 2 == 1
    return If(case[0], cases[1], Switch(*cases[2:]))

# Yeah, balancing all those parens isn't that bad.
# maybe this abstraction isn't worth it


#Switch(Aexpr.is_num(e), Aexpr.val(e),
#       Aexpr.is_add(e), Aexpr.val(e[0]), Aexpr.val(e[1])),
#       IntVal(42)
#       )

In [25]:
from z3 import *
def lemma(thm, by=[], admit=False):
    if admit:
        print("Admitting", thm)
    else:
        prove(Implies(And(by), thm))
    return thm

proved
proved
Admitting ForAll([x, y, z],
       append(x, append(y, z)) == append(append(x, y), z))


IndentationError: expected an indented block after class definition on line 4 (3081262896.py, line 7)

# Calculating Compilers


In [9]:
import kdrag as kd
import kdrag.smt as smt

Aexpr = smt.Datatype("Aexpr")
Aexpr.declare("num", ("val", smt.IntSort()))
Aexpr.declare("add", ("left", Aexpr), ("right", Aexpr))
Aexpr = Aexpr.create()

#kd.notation.induct = kd.utils.induct(Aexpr)
kd.utils.induct(Aexpr)

e = smt.Const("e", Aexpr)
eval = kd.kernel.define_fix("eval", [e], smt.IntSort(), 
                     lambda ev: kd.cond(
    (Aexpr.is_num(e), Aexpr.val(e)),
    (Aexpr.is_add(e), ev(e.left) + ev(e.right))
    ))


First define arithmetic expressions

In [30]:

Aexpr = Datatype("Aexpr")
Aexpr.declare("num", ("val", IntSort()))
Aexpr.declare("add", ("left", Aexpr), ("right", Aexpr))
Aexpr = Aexpr.create()
n = Int("n")
e, e1, e2 = Consts('e e1 e2', Aexpr)
def induct_aexpr(P):
    return Implies(And(        
        ForAll([n],     P(Aexpr.num(n))),
        ForAll([e1,e2], 
               Implies(And(P(e1), P(e2)), 
                       P(Aexpr.add(e1, e2))))),
        #-------------------------------------
        ForAll([e], P(e)))

eval = Function("eval", Aexpr, IntSort())
eval_def = ForAll([e], eval(e) == If(Aexpr.is_num(e), Aexpr.val(e),
                                     eval(Aexpr.left(e)) + eval(Aexpr.right(e))))

In [38]:
op = Datatype("op")
op.declare("add")
op.declare("push", ("val", IntSort()))
op = op.create()

def Option(s : SortRef):
    O = Datatype(f"Option_{s}")
    O.declare("none")
    O.declare("some", ("val", s))
    O = O.create()
    return O

IntList = List(IntSort())
Stack = IntList.t

exec = Function("exec", op, IntList.t, IntList.t) # option?
o = Const("o", op)
l = Const("l", IntList.t)
car = IntList.t.car
cdr = IntList.t.cdr
cons = IntList.t.cons
exec_def = ForAll([o,l], exec(o,l) == If(op.is_push(o), cons(op.val(o), l),
                                      #If(op.is_add(o),  
                                         cons(car(l) + car(cdr(l)), cdr(cdr(l)))))
                                     # )

Prog = List(op)
p = Const("p", Prog.t)
exec_prog = Function("exec_prog", Prog.t, IntList.t, IntList.t)
exec_prog_def = ForAll([p,l], exec_prog(p, l) == If(Prog.t.is_nil(p), l,
                                                  exec_prog(Prog.t.cdr(p), exec(Prog.t.car(p), l)))
                        )

compile = Function("compile", Aexpr, Prog.t)
compile_def = ForAll([e], compile(e) == If(Aexpr.is_num(e), Prog.t.cons(op.push(Aexpr.val(e)), Prog.t.nil),
                                           Prog.append(compile(Aexpr.left(e)), 
                                           Prog.append(compile(Aexpr.right(e)), 
                                                       Prog.t.cons(op.add, Prog.t.nil)))))




SyntaxError: incomplete input (2888385662.py, line 38)

In [9]:
type(IntList)

z3.z3.DatatypeSortRef

In [ ]:
class Nat():
    def __init__(self):
        pass
class Pos():
    
# Any is not closed and therefor doesn't have an indeuction principle of proving something true forall x:Any, unless it uses pure first order reasoning.
class Any():
    def __init__(self):
        self.t = DeclareSort("Any")
        self.sorts = {} # mapping from thing to Any injector,projector, tester
    def add_sort(s):
        self.sorts[s] = 
    def inj(self, sort):
        return Function(f"inj_{sort}", sort, self.t)
    def is_sort(self, sort):
        is_sort_def = ForAll([x], is_sort(x) == Exists([t], self.inj(sort)(t) == x))
        Function(f"is_{sort}", self.t, BoolSort())
    def injective(self):
        pass

class Map():
    pass
class Set():
    pass
class Enum():
    pass
class Record():
    pass

Nand 2 tetris

Maybe that's too agressive. How about a calculator

Quoted aexpr. hmm.
```ocaml
type aexpr =
 | Int of int
 | Plus of aexpr * aexpr
 | Quote of aexpr
```



```lean
-- import Mathlib
import Mathlib.Data.Nat.Basic
import Mathlib.Tactic.Basic
import Mathlib.Tactic.LibrarySearch
-- import Std.Data.List.Basic
-- import Std.Data.Nat.Basic
--open Nat
--#print Nat.
#eval 1 + 1
inductive aexpr where
  | num : ℕ → aexpr
--| var : string → aexpr
  |  add : aexpr → aexpr → aexpr

def eval : aexpr → ℕ
  | .num n => n
  | .add e1 e2 => eval e1 + eval e2

inductive op where
  | add : op
  | push : ℕ → op

-- [@reducible]
abbrev Prog := List op
abbrev Stack := List ℕ

def exec (p : Prog) (s : Stack) : Stack :=
  match p with
  | [] => s
  | (op.add :: p') =>
    match s with
    | (n1 :: n2 :: s') => exec p' ((n1 + n2) :: s')
    | _ => exec p' s
  | (op.push n :: p') => exec p' (n :: s)

def compile : aexpr → Prog
  | .num n => [op.push n]
  | .add e1 e2 => (compile e1) ++ (compile e2) ++ [op.add]
#print List
lemma exec_append : forall p1 p2 s, exec (p1 ++ p2) s = exec p2 (exec p1 s) := by
  intros p1 p2
  induction p1 with
  | nil => simp [exec]
  | cons h t ih =>
    simp [exec]
    cases h with
    | add =>
      intros s
      cases s with
      | nil => simp [exec, ih]
      | cons n1 s' =>
        cases s' with
        | nil => simp [exec, ih]
        | cons n2 s'' =>
          simp [exec]
          rw [ih]
    | push n =>
      simp [exec]
      intros s
      rw [ih]

theorem exec_compile : ∀ e s, exec (compile e) s = eval e :: s := by
  intros e
  induction e with
  | num n => simp [compile, exec, eval]
  | add e1 e2 IHe1 IHe2 =>
    intros s
    --simp [eval, compile, exec_append, exec]
    simp [eval]
    have : exec [op.add] ((eval e2) :: (eval e1) :: s) = (eval e1 + eval e2) :: s := by
      simp [exec, add_comm]
    rw [<- this]
    rw [<- IHe1]
    rw [<- IHe2]
    rw [<- exec_append]
    simp [compile]
    rw [exec_append]

/-
inductive fib : Nat -> Nat -> Type where
  | base0 : fib 0 0
  | base1 : fib 1 1
  | step  : n > 1 -> fib (n-1) a
                 -> fib (n-2) b -> fib n (a+b)

-/

```

```lean
-- import Mathlib
import Mathlib.Data.Nat.Basic
import Mathlib.Tactic.Basic
import Mathlib.Tactic.LibrarySearch
-- import Std.Data.List.Basic
-- import Std.Data.Nat.Basic
--open Nat
--#print Nat.
#eval 1 + 1
inductive aexpr where
  | num : ℕ → aexpr
--| var : string → aexpr
  |  add : aexpr → aexpr → aexpr

def eval : aexpr → ℕ
  | .num n => n
  | .add e1 e2 => eval e1 + eval e2

inductive op where
  | add : op
  | push : ℕ → op

-- [@reducible]
abbrev Prog := List op
abbrev Stack := List ℕ

def exec (p : Prog) (s : Stack) : Stack :=
  match p with
  | [] => s
  | (op.add :: p') =>
    match s with
    | (n1 :: n2 :: s') => exec p' ((n1 + n2) :: s')
    | _ => exec p' s
  | (op.push n :: p') => exec p' (n :: s)

def compile : aexpr → Prog
  | .num n => [op.push n]
  | .add e1 e2 => (compile e1) ++ (compile e2) ++ [op.add]
#print List
lemma exec_append : forall p1 p2 s, exec (p1 ++ p2) s = exec p2 (exec p1 s) := by
  intros p1 p2
  induction p1 with
  | nil => simp [exec]
  | cons h t ih =>
    simp [exec]
    cases h with
    | add =>
      intros s
      cases s with
      | nil => simp [exec, ih]
      | cons n1 s' =>
        cases s' with
        | nil => simp [exec, ih]
        | cons n2 s'' =>
          simp [exec]
          rw [ih]
    | push n =>
      simp [exec]
      intros s
      rw [ih]

theorem exec_compile : ∀ e s, exec (compile e) s = eval e :: s := by
  intros e
  induction e with
  | num n => simp [compile, exec, eval]
  | add e1 e2 IHe1 IHe2 =>
    intros s
    --simp [eval, compile, exec_append, exec]
    simp [eval]
    have : exec [op.add] ((eval e2) :: (eval e1) :: s) = (eval e1 + eval e2) :: s := by
      simp [exec, add_comm]
    rw [<- this]
    rw [<- IHe1]
    rw [<- IHe2]
    rw [<- exec_append]
    simp [compile]
    rw [exec_append]

/-
inductive fib : Nat -> Nat -> Type where
  | base0 : fib 0 0
  | base1 : fib 1 1
  | step  : n > 1 -> fib (n-1) a
                 -> fib (n-2) b -> fib n (a+b)

-/
```